## Import Libraries

In [373]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTENC

from xgboost import XGBClassifier
from sklearn.ensemble import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import *

## Import Data

In [374]:
# Reading Processed Train Set Data File
raw_d1 = pd.read_csv('Data/Processed_Trainset.csv')

# Feature Headers List
headers = list(raw_d1.columns)

# Reading Test Data Set
raw_d2 = pd.read_csv('Data/Processed_Testset.csv')

id_col = list(raw_d2.Client_ID)
raw_d2.drop(['Client_ID'], axis=1, inplace=True)

## Correlation

In [375]:
# Generating Correlation Matrix
X_T = np.array(raw_d1)[:,:-1]
corr_mat = pd.DataFrame(np.corrcoef(X_T.T), columns=headers[:-1], index=headers[:-1])
corr_mat.to_csv('Data/Correlation_Matrix.csv')

## Formatting

In [376]:
# Convert Data into ndArrays
train_data = np.array(raw_d1)
test_data = np.array(raw_d2)

# Total Training Data
X_tot = train_data[:, :-1]
Y_tot = train_data[:, -1]

# Splitting Data into a Training Set and a Cross Validation Set
X_train, X_cross, Y_train, Y_cross = train_test_split(X_tot, Y_tot, test_size = 0.2, random_state = 42, shuffle = True)
X_train.shape, Y_train.shape

((19200, 20), (19200,))

## SMOTE

In [377]:
# categorical_features = list(range(0,18))

# smote = SMOTENC(categorical_features, sampling_strategy='auto', k_neighbors=10, n_jobs=1)
# X_train,Y_train =  smote.fit_resample(X_train, Y_train)

# pd.DataFrame(np.concatenate((X_train,np.reshape(Y_train, (Y_train.shape[0], 1))), axis=1)
#              ,columns=headers).to_csv('Data/SMOTED.csv', index=None)
# X_train.shape, Y_train.shape

## Normalizing Data

In [378]:
# sc = RobustScaler()
# sc.fit(X_tot)

# X_tot = sc.fit_transform(X_tot)
# X_train = sc.transform(X_train)
# X_cross = sc.transform(X_cross)
# test_data = sc.transform(test_data)

# pd.DataFrame(X_tot, columns=headers[:-1])

## Classification Model 

In [379]:
rf = GradientBoostingClassifier(n_estimators= 300)

# rf = RandomForestClassifier(bootstrap=True, class_weight= None, max_depth= 8, max_features= 10, max_leaf_nodes= None,
#                             min_samples_leaf= 1, min_samples_split= 2, min_weight_fraction_leaf= 0.0, n_estimators= 300,
#                             n_jobs= 1, oob_score= False, random_state= 85, verbose= 0, warm_start= False)
# rf = XGBClassifier()

rf.fit(X_train, Y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=300,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

## Prediction and Evaluation

In [380]:
# Predicting Outcomes
predict_train = rf.predict(X_train)
predict_cross = rf.predict(X_cross)
predict_tot = rf.predict(X_tot)
predict_test = rf.predict(test_data)

# Computing Confusion Matrix on Cross Validation Set
confusion = confusion_matrix(Y_cross,predict_cross)

# Computing Accuracy Values
accuracy_train = accuracy_score(Y_train, predict_train)
accuracy_cross = accuracy_score(Y_cross, predict_cross)
accuracy_tot = accuracy_score(Y_tot, predict_tot)

# Computing AUC Values
auc_train = roc_auc_score(Y_train, predict_train)
auc_cross = roc_auc_score(Y_cross, predict_cross)
auc_tot = roc_auc_score(Y_tot, predict_tot)                

# Printing Accuracy Metrics
print("Train Accuracy            : {:.3f}%       AUC: {:.4f}".format(accuracy_train * 100, auc_train))
print("Cross Validation Accuracy : {:.3f}%       AUC: {:.4f} ".format(accuracy_cross * 100, auc_cross))
print("Total Set Accuracy        : {:.3f}%       AUC: {:.4f}".format(accuracy_tot * 100, auc_tot))
print("Confusion Matrix for Cross Validation Set :\n %s" % str(confusion))
print("Number of Ones Predicted for Test Data : %d" % np.sum(predict_test))

print(classification_report(Y_cross, predict_cross))

Train Accuracy            : 83.078%       AUC: 0.6730
Cross Validation Accuracy : 82.854%       AUC: 0.6668 
Total Set Accuracy        : 83.033%       AUC: 0.6718
Confusion Matrix for Cross Validation Set :
 [[3578  167]
 [ 656  399]]
Number of Ones Predicted for Test Data : 676
              precision    recall  f1-score   support

         0.0       0.85      0.96      0.90      3745
         1.0       0.70      0.38      0.49      1055

    accuracy                           0.83      4800
   macro avg       0.78      0.67      0.69      4800
weighted avg       0.81      0.83      0.81      4800



## Upload

In [382]:
# Uploading Predictions to a CSV
df = pd.DataFrame(predict_test, dtype=int)
df.index = id_col
df.to_csv('Data/Submission.csv', header=['NEXT_MONTH_DEFAULT'], index_label='Client_ID')